In [21]:
import pandas as pd
import numpy as np
from pathlib import Path
import requests

from domain.entities import Station
from services.data_fetcher import DataFetcher
from api.extractors import APIExtractor
from processing.validators import DataValidator
from processing.transformers import DataTransformer

def load_stations():
    csv_path = Path("data/stations/stations_meteo_transformees.csv")
    df = pd.read_csv(csv_path, sep=';')
    
    stations = []
    for _, row in df.iterrows():
        station = Station(
            id=row['id_nom'],
            name=row['nom'],
            longitude=row['longitude'],
            latitude=row['latitude']
        )
        stations.append(station)
    
    return stations

In [22]:
station = load_stations()[0]
station_id = load_stations()[0].id
station_id

'11-station-meteo-toulouse-soupetard'

In [23]:
url_base: str = "https://data.toulouse-metropole.fr/api/explore/v2.1/catalog/datasets/"
select: str = 'heure_de_paris, temperature_en_degre_c, humidite, pression'
url_final = f"{url_base + station_id}/records"
param = {
    'select': select,
    'where': """heure_de_paris >= now(days=-4)
        and minute(heure_de_paris) = 0
        and temperature_en_degre_c <= 50
        and temperature_en_degre_c >= -10
        and humidite >= 50
        and humidite < 100""",
    'order_by': 'heure_de_paris desc',
    'limit':'100'
}

data = requests.get(url_final, params=param)
data

<Response [200]>

In [26]:
extractor = APIExtractor()
data = extractor.extract(station)

In [27]:
data

,heure_de_paris,temperature_en_degre_c,humidite,pression
0,2025-11-06T11:00:00+00:00,12.6,94,98900
1,2025-11-06T08:00:00+00:00,13.5,93,99100
2,2025-11-06T03:00:00+00:00,17.2,80,98800
3,2025-11-06T02:00:00+00:00,17.4,80,98800
4,2025-11-03T23:00:00+00:00,8.1,92,100400
5,2025-11-03T11:00:00+00:00,14.4,88,100800
6,2025-11-03T07:00:00+00:00,9.1,94,100700
7,2025-11-03T05:00:00+00:00,9.5,93,100700
8,2025-11-03T04:00:00+00:00,10.7,93,100700
9,2025-11-03T03:00:00+00:00,11.4,93,100600


In [28]:
for _, row in data.iterrows():
    print(type(row))

<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>


In [25]:
transformer = DataTransformer()
formated_data = transformer.format_data(data)

TypeError: 'Response' object is not subscriptable

In [ ]:
validator = DataValidator()
validator.is_format_correct(formated_data)

False